### 데이터 다운로드

In [4]:
import urllib.request

In [6]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2", filename="kowiki-latest-pages-articles.xml.bz2")

('/tmp/tmpxdsvmpij', <http.client.HTTPMessage at 0x7fd0f7851390>)

### WikiExtractor 다운로드 및 설치

In [1]:
! git clone "https://github.com/attardi/wikiextractor.git"

Cloning into 'wikiextractor'...
remote: Enumerating objects: 733, done.
remote: Total 733 (delta 0), reused 0 (delta 0), pack-reused 733
Receiving objects: 100% (733/733), 1.28 MiB | 12.97 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [ ]:
# % cd ./wikiextractor
! python setup.py install

In [3]:
# ! pip install urllib

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [39]:
# 이미 준비된 extractor 사용
# % cd ./wikiextractor/
! python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 --json 
# ! python  -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 -o step1 --json -b 50M -q

INFO: Preprocessing 'kowiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Loaded 53554 templates in 362.7s
INFO: Starting page extraction from kowiki-latest-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (295.6 art/s)
INFO: Extracted 200000 articles (475.7 art/s)
INFO: Extracted 300000 articles (581.4 art/s)
INFO: Extracted 400000 articles (665.4 art/s)
INFO: Extracted 500000 articles (663.8 art/s)
INFO: Extracted 60000

### 하나의 텍스트 파일로 병합 후 전처리

In [74]:
output_dir = "text" 
dataset = []
filenames = list_wiki(output_dir)

In [75]:
filenames[:2]

['text/AA/wiki_00', 'text/AA/wiki_01']

In [76]:
# 예시로 몇줄 뽑아보기
i=0
with open(filenames[0], "r") as f:
    while True:
        line = f.readline()
        if line != '\n':
            i=i+1
            print("%d번째 줄 :"%i + line)
        if i==40:
            break 
    f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.

6번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

7번째 줄 :정계 입문.

8번째 줄 :1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

9번째 줄 :대통령 재임.

10번째 줄 :1976년 미합중국 (미국) 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 (당시 미국 대통령)를 누르고 당선되었다.

11번째 줄 :카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.

12번째 줄 :외교 정책.

13번째 줄 :카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.

14번째 줄 :그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.

15번째 줄 :카

In [ ]:
# Gensim 이용하기 - 데이터 다운로드부터 하는 듯하다.
# from gensim.corpora.wikicorpus import WikiCorpus

# wiki = WikiCorpus('kowiki-latest-pages-articles.xml.bz2', lemmatize=False)  # xml 파일을 넣어준다.

전체 문서 중 하나를 txt 파일로 저장

In [127]:
# lxml을 이용해 xml 파일 읽기
# 와.... default form이 xml이 아님 ;;
#  -> 단순 \n로 여러 document가 연결된 게 문제인지?
import lxml.etree as etree

tree = etree.parse(filenames[0])
root = tree.getroot()
    
## 저장할 txt 파일 오픈

# # root.findall('doc')  # Tag name, first level only
# docs = root.findall('.//doc')  # XPath, recursive
# for doc in docs:
#     for element in doc.findall('.//seg'):
#         tfile.write(element.text.strip() + '\n')  # strip: 앞뒤 trim

XMLSyntaxError: ignored

In [81]:
# 문자열에 조건 걸어서 <doc>, <\doc> 태그를 제외한 나머지 txt를 연결하여 저장
data_str = ''

with open(filenames[0], "r") as f:
    lines = f.readlines()
    # line = f.read()

    # for line in f:
    # line = line.strip() # 주어진 문자열에서 양쪽 끝에 있는 공백과 \n 기호를 삭제시켜 준다

    for line in lines:
        if line:
            if line is not '\n' and not 'doc' in line: # 첫번째 조건 = 연속된 개행문자 제거, 두번째 조건 = 태그 제거
                dataset.append(line)
                data_str += line
            
            # print(line)
            # tmp = trim_text(line)
            # dataset.append(tmp)
            # print(type(tmp))
            # f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
            # f.write("\n\n\n\n") # 구분자

In [79]:
dataset[:4]

['지미 카터\n',
 '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n',
 '생애.\n',
 '어린 시절.\n']

In [83]:
data_str[:100]

'지미 카터\n제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬'

In [84]:
output = f"{output_dir}/wiki_data.txt"

with open(output, "w") as f:
    f.write(data_str)

전체 문서를 txt 파일로 저장

In [85]:
data_str = ''

for filename in filenames:
    with open(filename, "r") as f:
        lines = f.readlines()

        for line in lines:
            if line:
                if line is not '\n' and not 'doc' in line: # 첫번째 조건 = 연속된 개행문자 제거, 두번째 조건 = 태그 제거
                    dataset.append(line)
                    data_str += line

In [98]:
data_str[909000:915900]

'당시에, 그는 Frans G. Bengtsson의 역사 소설 "The Long Ships"을 읽고 있었다고 한다. 이 책은 바이킹과 하랄드 블라톤 국왕의 이야기가 적혀있었다. 하랄드 블라톤 왕이 스칸디나비아를 통일했던 것처럼, \'난립하는 여러 무선 통신 규격을 통합하자\'는 염원이 담겼다고 한다.\n블루투스의 로고는, 북유럽의 룬 문자(옐링 거석의 비석에서 찾을 수 있는)로 하랄드 블라톤 국왕의 이름의 앞글자 \xa0 (H)와 \xa0(B)를 결합한 모양에서 유래한다.\n사양과 특징.\nPANs의 산업 표준중 하나인 블루투스는 ISM 대역인 2.45GHz를 사용한다. 버전 1.1과 1.2의 경우 속도가 723.1kbps에 달하며, 버전 2.0의 경우 EDR을 특징으로 하는데, 이를 통해 2.1Mbps의 속도를 낼 수 있다. 블루투스는 RS-232, 유선 USB를 대체하는 개념이며 이는 와이파이가 이더넷을 대체하는 개념과 유사하다. 암호화에는 SAFER을 사용한다. 장치끼리 믿음직한 연결을 성립하려면 키워드를 이용한 페어링이 이루어지는데, 이 과정이 없는 경우도 있다.\n블루투스 1.0과 1.0B.\n1.0과 1.0B는 많은 문제점을 가지고 있고 다양한 제조사들이 그들의 제품 간의 상호 호환성을 가지게 하는 데에 많은 어려움을 겪었다. 1.0과 1.0B는 또한 핸드셰이킹 과정에서 블루투스 하드웨어 장치 주소(BD_ADDR)를 반드시 전송해야 하므로 프로토콜 수준에서의 익명(IP와 같은 주소 없이)의 연결(rendering anonymity)을 할 수 없었는데, 이는 블루투스 환경에서 제공 되도록 계획한 소비확대정책(Consumerium)같은 서비스를 제공하는 데에 큰 제한사항이었다. 블루투스는 2.4Ghz의 주파수를 사용하고 무선 랜(802.11b/g/n)도 2.4Ghz대의 주파수를 사용한다. 같은 주파수를 사용하므로 동시사용에 따른 충돌은 피할 수 없다. 양 제품의 초기 보급 시에는 그것에 대한 우려의 목소리가 높았다. 하지만 블루투스의 버전업과 연결 특성

In [100]:
output = f"{output_dir}/wiki_data.txt"

with open(output, "w") as f:
    f.write(data_str)

In [ ]:
# for filename in filenames:
#     with open(filename, "r") as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 dataset.append(trim_text(line))
#                 # f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
#                 # f.write("\n\n\n\n") # 구분자

# ------------ 
# import pandas as pd

# in_file = "<path of input>/kowiki_yyyymmdd.csv"
# out_file = "<path of output>/kowiki.txt"
# SEPARATOR = u"\u241D"
# df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
# with open(out_file, "w") as f:
#   for index, row in df.iterrows():
#     f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
#     f.write("\n\n\n\n") # 구분자

# ------------ 
# paul-shin 출처
# 자장파일 결정
# now = datetime.datetime.now().strftime("%Y%m%d")
# output = f"{output_dir}/kowiki.csv"
output = f"{output_dir}/wiki_data.txt"

# 위키저장 (csv)
# if 0 < len(dataset):
#     df = pd.DataFrame(data=dataset)
#     df.to_csv(output, sep=SEPARATOR, index=False)

# # 파일삭제
# del_garbage(output_dir)

### SentencePiece 학습

In [102]:
! pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.5MB/s 


In [106]:
import sentencepiece as spm
import os

corpus = os.path.join(output_dir, "wiki_data.txt")
prefix = "kowiki"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [126]:
# vocab, model이 생성된 것을 확인해야 한다.
# 간단한 테스트
vocab_file = "./kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "와 이 영화 꿀잼 ㅋㅋㅋ",
  "정말 재미 없는 영화로군요.",
  "세기의 명작!",
  "겨울 날씨가 너무 춥죠?",
  "이 원소는 원자량 16를 가집니다. 탄소 보단 무겁습니다.",
  "원자량은 특정 환경 하에서 해당 원소의 동위 원소의 존재 비율에 따라 가중평균한 질량을 말한다. "
]

for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

와 이 영화 꿀잼 ㅋㅋㅋ
['▁와', '▁이', '▁영화', '▁', '꿀', '잼', '▁', 'ᄏ', 'ᄏ', 'ᄏ']
[771, 8, 297, 3520, 4751, 5507, 3520, 7076, 7076, 7076]

정말 재미 없는 영화로군요.
['▁정', '말', '▁재', '미', '▁없는', '▁영화', '로', '군', '요', '.']
[35, 3763, 171, 3609, 908, 297, 3527, 3647, 3684, 3523]

세기의 명작!
['▁세', '기의', '▁명', '작', '!']
[75, 591, 162, 3654, 4140]

겨울 날씨가 너무 춥죠?
['▁겨울', '▁날', '씨', '가', '▁너무', '▁', '춥', '죠', '?']
[3144, 707, 3953, 3532, 2914, 3520, 6488, 4736, 4188]

이 원소는 원자량 16를 가집니다. 탄소 보단 무겁습니다.
['▁이', '▁원', '소는', '▁원자', '량', '▁16', '를', '▁가', '집', '니다', '.', '▁탄', '소', '▁보', '단', '▁무', '겁', '습니다', '.']
[8, 126, 2161, 3069, 3878, 366, 3550, 23, 3800, 1180, 3523, 821, 3613, 47, 3660, 107, 4701, 2565, 3523]

원자량은 특정 환경 하에서 해당 원소의 동위 원소의 존재 비율에 따라 가중평균한 질량을 말한다. 
['▁원자', '량은', '▁특정', '▁환경', '▁하', '에서', '▁해당', '▁원', '소의', '▁동', '위', '▁원', '소의', '▁존재', '▁비', '율', '에', '▁따라', '▁가', '중', '평', '균', '한', '▁질', '량을', '▁말한다', '.']
[3069, 2709, 2288, 1547, 27, 10, 910, 126, 2008, 57, 3587, 126, 2008, 538, 78, 4032, 3524, 27